Importing Packages

In [2]:
import numpy as np
import pandas as pd
import difflib #get the close match
from sklearn.feature_extraction.text import TfidfVectorizer # convert text to numerical
from sklearn.metrics.pairwise import cosine_similarity

Data Collection and Pre-Processing

In [4]:
# loading data
movies_data = pd.read_csv("/content/movies.csv")
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
# number of rows and columns in dataset
movies_data.shape

(4803, 24)

In [7]:
# selecting relevent features for recommendation AKA feature selection -- selecting features based on the type of system you want to recommend. in this case we want a content and popularity system
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
print(selected_features)

## you dont need all of the columns only select what you think will be necessary to make a recommendation

['genres', 'keywords', 'tagline', 'cast', 'director']


In [8]:
# replacing numm values with null string
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('') # replacing null values with empty string

In [10]:
# combining all the 5 selected features
combined_features = movies_data['genres']+' '+ movies_data['keywords']+' '+movies_data['tagline']+' '+movies_data['cast']+' '+['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [11]:
# converting the text data to numerical feature vectors
vectorizer = TfidfVectorizer()

In [12]:
# tranform data to numerical values
feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
print(feature_vectors)

  (0, 3951)	0.03278705966723603
  (0, 11944)	0.21783692407157268
  (0, 9358)	0.16211904900345359
  (0, 8035)	0.2425508851315671
  (0, 13376)	0.16788598677970765
  (0, 15268)	0.20032592973815155
  (0, 12876)	0.20792602308921068
  (0, 12211)	0.21982462716437176
  (0, 15836)	0.20390625343942767
  (0, 15585)	0.2386891257137131
  (0, 12235)	0.1614214256709989
  (0, 10522)	0.2747093923734331
  (0, 10234)	0.09135661405936532
  (0, 15576)	0.12943593761516567
  (0, 13980)	0.07163536492433677
  (0, 4570)	0.2425508851315671
  (0, 13069)	0.21596286774651774
  (0, 2965)	0.25198313440623776
  (0, 15189)	0.12669169288916793
  (0, 13166)	0.3428679840722249
  (0, 5381)	0.16624629213634265
  (0, 2817)	0.2242027289018011
  (0, 3392)	0.21596286774651774
  (0, 5022)	0.10463026444035904
  (0, 12451)	0.10463026444035904
  :	:
  (4801, 385)	0.20147675061976653
  (4801, 6373)	0.3164734698991806
  (4801, 10669)	0.2379294748661469
  (4801, 1533)	0.17470057708349773
  (4801, 10000)	0.14807918205096432
  (4801, 68

Find Similiarity score with Cosine Similiarity Score

In [14]:
# getting similiarity score using cosine similiarity
similarity = cosine_similarity(feature_vectors)

In [15]:
print(similarity)

[[1.         0.07669359 0.01635542 ... 0.00111017 0.00123842 0.00126877]
 [0.07669359 1.         0.0361322  ... 0.03883269 0.00111246 0.00113973]
 [0.01635542 0.0361322  1.         ... 0.00115905 0.03391284 0.00132463]
 ...
 [0.00111017 0.03883269 0.00115905 ... 1.         0.00127895 0.03674788]
 [0.00123842 0.00111246 0.03391284 ... 0.00127895 1.         0.00146166]
 [0.00126877 0.00113973 0.00132463 ... 0.03674788 0.00146166 1.        ]]


In [16]:
print(similarity.shape)

(4803, 4803)


In [17]:
# getting the movie name from the user

movie_name = input(' Enter your favorite movie name: ')

 Enter your favorite movie name: iron man


In [21]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data["title"].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [22]:
# finding the close match for the movie name given by a user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [23]:
close_match = find_close_match[0]
print(close_match)

Iron Man


In [25]:
# find index of the movie with title

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]
print(index_of_the_movie)

# batman index is 68, this number will change with each movie input

68


In [27]:
# getting list of similiar movies based on index value
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)
# the numbers that print are index of movie, similiarity score compared to iron man movie

[(0, 0.03584206137753142), (1, 0.05924876249043993), (2, 0.015607064769752065), (3, 0.008087105405779694), (4, 0.03579200161452046), (5, 0.015922979428965603), (6, 0.05408550049277217), (7, 0.26589390805703095), (8, 0.009451578491926717), (9, 0.083308976018144), (10, 0.08255960974826987), (11, 0.013548574105235718), (12, 0.0159328163829399), (13, 0.014212641839413285), (14, 0.10558233740663243), (15, 0.008812855210279753), (16, 0.2504792083437875), (17, 0.014828331412129058), (18, 0.04509237309961639), (19, 0.08274987472185044), (20, 0.08599830274327105), (21, 0.01261735938747427), (22, 0.008092239158986735), (23, 0.007807907316798083), (24, 0.014219468783616145), (25, 0.0010446208359733145), (26, 0.24015014844867336), (27, 0.0331909216328085), (28, 0.07083080733691056), (29, 0.01599875097703665), (30, 0.08685556152090212), (31, 0.3371368461537161), (32, 0.03127708242550444), (33, 0.1422847129822525), (34, 0.0010787337361456812), (35, 0.03854151344758508), (36, 0.03443383573711991), (3

In [28]:
len(similarity_score)

4803

In [31]:
# sorting movies based on their similiarity score

sorted_similiar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) # the labmda values = the second number in the similiarity score
print(sorted_similiar_movies)

[(68, 0.9999999999999999), (79, 0.37027467830730154), (31, 0.3371368461537161), (7, 0.26589390805703095), (16, 0.2504792083437875), (26, 0.24015014844867336), (85, 0.22329610532227587), (182, 0.20812094278500717), (511, 0.18007886345409335), (64, 0.16418320665280903), (203, 0.15828343903957937), (174, 0.1562900086991977), (101, 0.15397795236414022), (46, 0.15232915634207414), (4401, 0.1522120228242857), (169, 0.14756856069280094), (94, 0.1452244121541296), (788, 0.1435382506705262), (33, 0.1422847129822525), (126, 0.139835129339762), (783, 0.13035502526307383), (138, 0.12720508663497043), (607, 0.12569683547217136), (353, 0.1252607215428451), (2442, 0.1239366854809223), (38, 0.12126023196544766), (1740, 0.12072920851652377), (122, 0.11866755500774528), (242, 0.11623903632438638), (1451, 0.11462850627771107), (1553, 0.11451164964542083), (954, 0.11320730388865538), (618, 0.11204238090457046), (3166, 0.10941859109865319), (1210, 0.10906918066715098), (720, 0.10843148166310577), (2235, 0.

In [33]:
# print the name of similiar movies based on the index

print('Movies suggested for you : \n')
i=1

for movie in sorted_similiar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30): # number of movies we want to retrieve for our user
    print(i, '.',title_from_index)
    i+=1 # each time for loop runs it increases by the next number starting with 1

Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . X-Men: Apocalypse
11 . X2
12 . The Incredible Hulk
13 . X-Men: First Class
14 . X-Men: Days of Future Past
15 . The Helix... Loaded
16 . Captain America: The First Avenger
17 . Guardians of the Galaxy
18 . Deadpool
19 . X-Men: The Last Stand
20 . Thor: The Dark World
21 . Mortdecai
22 . The Last Airbender
23 . Sky Captain and the World of Tomorrow
24 . Tropic Thunder
25 . Southland Tales
26 . The Amazing Spider-Man 2
27 . Kick-Ass 2
28 . X-Men Origins: Wolverine
29 . Fantastic Four


Movie Recommendation System

In [36]:
movie_name = input(' Enter your favorite movie name: ')

list_of_all_titles = movies_data["title"].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similiar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')
i=1

for movie in sorted_similiar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1 # each time for loop runs it increases by the next number starting with 1

 Enter your favorite movie name: avatar
Movies suggested for you : 

1 . Avatar
2 . Alien
3 . Guardians of the Galaxy
4 . Star Trek Beyond
5 . Galaxy Quest
6 . Star Trek Into Darkness
7 . Aliens
8 . Gravity
9 . Cargo
10 . Alien³
11 . Moonraker
12 . Trekkies
13 . Gettysburg
14 . Pocahontas
15 . Space Dogs
16 . The Helix... Loaded
17 . Event Horizon
18 . Jason X
19 . Clash of the Titans
20 . The Right Stuff
21 . Space Cowboys
22 . Shadow Conspiracy
23 . The Astronaut's Wife
24 . Terminator Salvation
25 . Planet of the Apes
26 . Star Trek
27 . Star Wars: Clone Wars: Volume 1
28 . Wing Commander
29 . Sunshine
